# Rede Neural
A rede neural que irá identificar se um span (conjunto de palavras) é:
 - um endereço ou uma localização
 - ou não é nenhum ponto de interesse.
 

In [23]:
from google.colab import drive
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import spacy
import numpy as np
import math   #Se não importar o math o keras.callbacks da erro
from tensorboard.plugins.hparams import api as hp
import random
import re
# Load the TensorBoard notebook extension
%load_ext tensorboard
drive.mount('/content/drive')
HPARAMS = {'num_filters1': 64,
           'num_filters2': 64,
           'num_words1': 3,
           'num_words2': 3,
           'hp_dropout': 0.2,
           'hp_optimizer': 'adam',
           'dense_units': 8}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
cd drive/My Drive/Mestrado Unicamp/Hackaton CNJ

[Errno 2] No such file or directory: 'drive/My Drive/Mestrado Unicamp/Hackaton CNJ'
/content/drive/My Drive/Mestrado Unicamp/Hackaton CNJ


## Baixando dados, tokenização, criação do word_to_index_dict

word_to_index_dict é um dicionário que converte uma palavra para um integer que representa aquela palavra.

word_to_index_dict_inverse faz o inverso.

In [25]:
max_seq_len = 20
with open("incluir_address.txt", "r", encoding="unicode-escape") as f:
  address = f.readlines()
with open("address_syntetic.txt", "r", encoding="unicode-escape") as f:
  address = address + f.readlines()
address = [i.replace('\n','') for i in address]
address = [i.replace(',', ' , ') for i in address]
address = [i.replace(';', ' ; ') for i in address]
print('len(address) = ', len(address))
with open("incluir_not_address.txt", "r", encoding="unicode-escape") as f:
  not_address = f.readlines()
with open("not_address_syntetic.txt", "r", encoding="unicode-escape") as f:
  not_address = not_address + f.readlines()
not_address = [i.replace('\n','') for i in not_address]
not_address = [i.replace(',', ' , ') for i in not_address]
not_address = [i.replace(';', ' ; ') for i in not_address]
print('len(not_address) = ', len(not_address))

#make the same size
if len(address) > len(not_address):
  address = address[:len(not_address)]
else:
  not_address = not_address[:len(address)]

lista_de_frases = address+not_address
vocab_size = 64730
tokenizer = Tokenizer(num_words=vocab_size, lower=True, split=' ',
                      char_level=False, oov_token="<UNK>", document_count=0)

lista_de_lista_de_palavras = [text_to_word_sequence(frase, filters = '!"“”#$%&()*+-/:=?@[\\]^_`{|}~\t\n', split=' ') for frase in lista_de_frases]

tokenizer.fit_on_texts([palavra for lista_de_palavras in lista_de_lista_de_palavras for palavra in lista_de_palavras])
word_to_index_dict = tokenizer.word_index
word_to_index_dict[','] = len(word_to_index_dict)
word_to_index_dict[';'] = len(word_to_index_dict)
word_to_index_dict['<PAD>'] = 0
print("len(word_to_index_dict) =", len(word_to_index_dict))
#texto_lista = [[word_to_index_dict[word] for word in frase] for frase in texto_lista]

len(address) =  29745
len(not_address) =  33027
len(word_to_index_dict) = 39474


A função _index_list_to_text transforma: array([ 0, 39, 72, 73, 74, 10, 40,  6, 25, 75], dtype=int32) em "<PAD> ela estava aguardando impaciente a chegada de seus filhotinhos" . Serve mais para testes.

In [26]:
def _index_list_to_text(index_list, word_index_inverse):
    return ' '.join([word_index_inverse.get(i, "") for i in index_list])

## Preparando o dicionário e os pesos da camada densa

In [27]:
embedding_size = 50
embedding_file = "glove_s50.txt"
def _load_embeddings_into_memmory(embedding_file, embedding_size):
    embeddings_index = dict()
    f = open(embedding_file, encoding="utf8")
    for line in f:
        try:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except:
            print(f"Word not included:{values[0]}")
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    print()
    #Remover embeddings errados
    key_to_remove=[]
    for key,value in embeddings_index.items():
        if len(value)!=embedding_size:
            key_to_remove.append(key)
    for key in key_to_remove:
        del embeddings_index[key]
    print("Removeu os seguintes tokens errados:", key_to_remove)
    return embeddings_index

In [28]:
def _add_required_tokens_to_embeddings_index(required_tokens_list, embeddings_index, embedding_size):
  #Inicializa os required_tokens randomicamente
    if required_tokens_list != []:
        for required_token in required_tokens_list:
            embeddings_index[required_token] = [random.uniform(-1,1) for iter in range(embedding_size)]
    return embeddings_index

In [29]:
embeddings_index = _load_embeddings_into_memmory(embedding_file, embedding_size)
embeddings_index= _add_required_tokens_to_embeddings_index(['<PAD>', '<UNK>'], embeddings_index, embedding_size)
print(len(embeddings_index))
embeddings_index['casa']

Word not included:r$
Word not included:00
Word not included:三藏法師玄奘奉
Word not included:r$
Loaded 929594 word vectors.

Removeu os seguintes tokens errados: ['929605', '00', '0', 'r$', '-0.307602']
929591


array([-8.087840e-01, -2.344940e-01,  6.837990e-01, -3.552256e+00,
        2.660380e-01,  1.764880e-01, -8.301800e-02, -1.756600e+00,
        5.187320e-01,  7.874550e-01, -2.560460e-01,  5.267100e-01,
        2.626010e-01,  1.581000e-02,  4.528480e-01, -4.253290e-01,
       -1.091950e-01, -3.496000e-02, -3.325440e-01,  1.537287e+00,
        3.229070e-01, -6.341390e-01,  2.590570e-01, -3.703710e-01,
       -5.205270e-01, -4.843610e-01, -6.838160e-01, -9.480820e-01,
       -4.697990e-01,  1.852468e+00,  7.642130e-01,  2.127000e-03,
        2.985810e-01, -8.077000e-02, -3.647550e-01,  3.420010e-01,
        1.744480e-01, -5.170710e-01, -9.487810e-01,  8.373230e-01,
       -2.660080e-01,  3.273090e-01,  4.745350e-01, -5.496020e-01,
        2.931430e-01,  1.284411e+00, -6.722870e-01,  2.427270e-01,
        9.723310e-01, -7.579990e-01], dtype=float32)

A função _create_embedding_weights vai criar os pesos que serão usados na camada Embeddings.

Se use_full_embeddings_index for False, os pesos só existiram para palavras que existem nas bases de treino e teste, fornecidas pelo word_to_index_dict.

Se use_full_embeddings_index for True, todos os pesos do arquivo de embeddings serão mantidos. Mas vou criar novo word_to_index_dict, ou seja, preciso remapear x_train, x_test.

In [30]:
def _create_embedding_weights(embedding_size, word_to_index_dict, embeddings_index, use_full_embeddings_index = False):
    if use_full_embeddings_index:
      embedding_weights = np.zeros((len(embeddings_index), embedding_size))
      print(embedding_weights.shape)
      new_word_to_index_dict = {}
      for i, (word, dense_vector) in enumerate(embeddings_index.items()):
        new_word_to_index_dict[word] = i
        embedding_weights[i, :] = embeddings_index[word]
      return embedding_weights, new_word_to_index_dict

    else:
      embedding_weights = np.zeros((len(word_to_index_dict), embedding_size))
      print(embedding_weights.shape)
      for word, i in word_to_index_dict.items():
          try:
              embedding_weights[i, :] = embeddings_index[word]
          except KeyError:
              # Para os tokens que não houver peso no GLOVE, inializar randomicamente
              embedding_weights[i, :] = [random.uniform(-1,1) for iter in range(embedding_size)]
      return embedding_weights, word_to_index_dict

In [31]:
def incluir_palavras_de_reservas():
  with open("palavras_para_acrescentar.txt", "r", encoding="utf8") as f:
    palavras_para_acrescentar = f.readlines()
  for palavra in palavras_para_acrescentar:
    if not palavra in word_to_index_dict:
      word_to_index_dict[palavra] = word_to_index_dict['ianomamis']

embedding_weights, word_to_index_dict = _create_embedding_weights(embedding_size, word_to_index_dict, embeddings_index, use_full_embeddings_index = True)
print('len(word_to_index_dict) =', len(word_to_index_dict))
incluir_palavras_de_reservas()
print('len(word_to_index_dict) =', len(word_to_index_dict))

word_to_index_dict_inverse =  {w: k for k, w in word_to_index_dict.items()}
vocab_size = len(embedding_weights)
print('len(embedding_weights) =', len(embedding_weights))

(929591, 50)
len(word_to_index_dict) = 929591
len(word_to_index_dict) = 930344
len(embedding_weights) = 929591


In [32]:
pickle_file = open("word_to_index_dict.pkl", "wb")
pickle.dump(word_to_index_dict, pickle_file)
pickle_file.close()
#pickle_file = open("word_to_index_dict.pkl", "rb")
#dic = pickle.load(pickle_file)
#pickle_file.close()

In [33]:
#Testando se a implementação acima está correta
word = '<PAD>'
print(embeddings_index[word])
embedding_weights[word_to_index_dict[word]] - embeddings_index[word]

[0.7976662943830599, 0.5847921107243181, -0.4254656528312295, -0.797669121158094, 0.5622590622047146, 0.33737460274072895, -0.34261223068389723, -0.33825808575478966, 0.3160842127751886, 0.7774339545666533, -0.1721633298249734, -0.8425803571185067, 0.9728765497400131, -0.18669337182089807, 0.3510107196855301, -0.7845197259551795, -0.9470202069467994, -0.4659581697664281, 0.3744076231554474, -0.08701405400508633, 0.671297759628471, 0.14007163312652415, 0.9228761525924829, 0.39072392678998735, 0.5026359986915532, -0.040437159890612806, 0.44246195021773715, -0.10638495977647255, -0.9695325339246492, 0.6153759378443671, -0.5636049632649585, 0.9199797302993655, 0.9351561001307185, -0.572313564155905, -0.332871870456231, 0.06913303146352634, 0.6415581514325606, -0.35474717057717053, 0.7747299330650947, 0.8059627212223202, -0.18013130863289062, -0.39306658804925054, -0.9126750056474793, -0.8037563295728058, 0.5769755031064323, -0.4104569069568922, -0.4777874554142394, -0.6145204957435861, -0.

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Transformando as Listas de palavras em Listas de Índices e X e y

In [34]:
def prepararInput(lista_de_frases):
  lista_de_lista_de_palavras = [text_to_word_sequence(frase, filters = '!"“”#$%&()*+-/:=?@[\\]^_`{|}~\t\n', split=' ') for frase in lista_de_frases]
  for i in range(len(lista_de_lista_de_palavras)):
    for j in range(len(lista_de_lista_de_palavras[i])):
      lista_de_lista_de_palavras[i][j] = re.sub("[0-9]+", 'cinco', lista_de_lista_de_palavras[i][j])

  lista_de_lista_de_indices = [[word_to_index_dict[palavra] if palavra in word_to_index_dict else word_to_index_dict['<UNK>'] for palavra in lista_de_palavras] for lista_de_palavras in lista_de_lista_de_palavras]

  x = lista_de_lista_de_indices

  x = pad_sequences(x, 
                    maxlen=max_seq_len, 
                    dtype='int32',
                    padding='pre', 
                    truncating='pre', 
                    value=word_to_index_dict['<PAD>'])
  return x
x = prepararInput(lista_de_frases)

y = np.append(np.ones(int(len(x)/2)), np.zeros(int(len(x)/2)))
print('len(x) =', len(x))
print('len(y) =', len(y))
DATASET_SIZE = len(x)

len(x) = 59490
len(y) = 59490


In [35]:
# Testando para ver se os dados estão bem construídos
for i in range(20):
    print('data:                      ', lista_de_frases[i])
    print('lista_de_lista_de_palavras:', lista_de_lista_de_palavras[i])
    #print('lista_de_lista_de_indices: ', _index_list_to_text(lista_de_lista_de_indices[i], word_to_index_dict_inverse))
    print('x      :                   ', _index_list_to_text(x[i], word_to_index_dict_inverse))
    print()


data:                       Aldeia AikanÃ£ ,  na Reserva IndÃ­gena TubarÃ£o LatundÃª.
lista_de_lista_de_palavras: ['aldeia', 'aikanã£', ',', 'na', 'reserva', 'indã\xadgena', 'tubarã£o', 'latundãª.']
x      :                    <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> aldeia <UNK> , na reserva indã­gena tubarã£o <UNK>

data:                       Reserva IndÃ­gena TubarÃ£o LatundÃª.
lista_de_lista_de_palavras: ['reserva', 'indã\xadgena', 'tubarã£o', 'latundãª.']
x      :                    <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> reserva indã­gena tubarã£o <UNK>

data:                       Terra IndÃ­gena MequÃ©ns
lista_de_lista_de_palavras: ['terra', 'indã\xadgena', 'mequã©ns']
x      :                    <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> terra indã­gena <UNK>

data:                       reserva indÃ­gena Alto TuriaÃ§u
lista_de_list

## Construindo o Dataset

In [36]:

dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=DATASET_SIZE)

train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.10 * DATASET_SIZE)
test_size = int(0.10 * DATASET_SIZE)

train_dataset = dataset.take(train_size).batch(20).prefetch(1)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size).batch(20).prefetch(1)
test_dataset = dataset.take(test_size).batch(20).prefetch(1)

In [37]:
# Testando para ver se o dataset foi bem construído
a = list(test_dataset.as_numpy_iterator())
batch = 0
for i in range(10):
  print('_index_list_to_text = ', _index_list_to_text(list(a[batch][0][i]), word_to_index_dict_inverse))
  print('y =', a[batch][1][i])

_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> rua ambrosina felix , cinco fns , barroquinha
y = 1.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> rua doutor dantas junior , nº cinco , itapicuru
y = 1.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> rua otavio dias de oliveira , sn , quadra cinco lote cinco , bairro trindade
y = 1.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> ideal para reprodução de vídeos para o
y = 0.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> rua das dalias sn , alpinópolis mg cep cinco cinco
y = 1.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> cinco cinco cinco cinco cinco cinco cinco cinco cinco
y = 0.0
_index_list_to_text =  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

## Modelos

In [38]:
def CNN_deep(vocab_size, embedding_size, embedding_weights, input_length, hparams):
    
    NUM_FILTERS1, NUM_FILTERS2,NUM_WORDS1, NUM_WORDS2,HP_DROPOUT, HP_OPTIMIZER, DENSE_UNITS = \
    "num_filters1", "num_filters2", "num_words1", "num_words2", "hp_dropout", 'hp_optimizer', 'dense_units'
    
    inputs = keras.layers.Input(shape=(input_length,))
    x = keras.layers.Embedding(vocab_size,
                                embedding_size,
                                input_length=input_length,
                                weights=[embedding_weights],
                                trainable=False)(inputs)
    x = keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(x)
    x = keras.layers.Conv1D(hparams[NUM_FILTERS1], kernel_size=hparams[NUM_WORDS1],
                                            activation="relu")(x)
    x = keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(x)
    x = keras.layers.MaxPooling1D()(x)

    x = keras.layers.Conv1D(hparams[NUM_FILTERS2], kernel_size=hparams[NUM_WORDS1],
                                            activation="relu")(x)
    x = keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(x)
    x = keras.layers.MaxPooling1D()(x)

    x = keras.layers.Conv1D(hparams[NUM_FILTERS2], kernel_size=hparams[NUM_WORDS1],
                                            activation="relu")(x)
    x = keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(x)
    x = keras.layers.MaxPooling1D()(x)

    x = keras.layers.Conv1D(hparams[NUM_FILTERS2], kernel_size=hparams[NUM_WORDS2],
                                            activation="relu")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dense(hparams[DENSE_UNITS], activation='relu')(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)
    outputs = x
    model = keras.Model(inputs=inputs, outputs=outputs, name='CNN_Deep_Functional_API')
    return model
selectedModel = CNN_deep

In [39]:
def biLSTM(vocab_size, embedding_size, embedding_weights, input_length, hparams):
    
    NUM_FILTERS1, NUM_FILTERS2,NUM_WORDS1, NUM_WORDS2,HP_DROPOUT, HP_OPTIMIZER, DENSE_UNITS = \
    "num_filters1", "num_filters2", "num_words1", "num_words2", "hp_dropout", 'hp_optimizer', 'dense_units'
    
    inputs = keras.layers.Input(shape=(input_length,))
    x = keras.layers.Embedding(vocab_size,
                                embedding_size,
                                input_length=input_length,
                                weights=[embedding_weights],
                                trainable=False)(inputs)
    x = keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(x)

    x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True), input_shape=(200, 100))(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dense(hparams[DENSE_UNITS], activation='relu')(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)
    outputs = x
    model = keras.Model(inputs=inputs, outputs=outputs, name='biLSTM')
    return model
selectedModel = biLSTM

In [40]:
def trainer(model, train_dataset, val_dataset, hparams, max_epochs=100, verbose=True):
    BATCH_SIZE=16
    
    TOTAL_EPOCH_COUNT = 100
    PATIENCE=4
    LOGDIR="training_logs"
    callbacks=[tf.keras.callbacks.TensorBoard(LOGDIR),
               hp.KerasCallback(LOGDIR, hparams),
               keras.callbacks.EarlyStopping(monitor='accuracy',
                                             patience=PATIENCE,
                                             restore_best_weights=True)]
    
    model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics=['accuracy'])
    model.summary()
    model.fit(train_dataset,
                      batch_size=BATCH_SIZE,
                      epochs=max_epochs,
                      shuffle=True,
                      validation_data=val_dataset,
                      verbose=verbose,
                      callbacks=callbacks)
    
    test_score, test_accuracy = model.evaluate(val_dataset, batch_size=BATCH_SIZE,verbose=0)
    if verbose == False:
        train_score, train_accuracy = model.evaluate(train_dataset, batch_size=BATCH_SIZE,verbose=0)
        print(f'Train score:{train_score}, Train accuracy: {train_accuracy}.')
        print(f'Test score:{test_score}, Test accuracy: {test_accuracy}.')
    return test_accuracy

# Training and saving

In [41]:
model = selectedModel(vocab_size = vocab_size,
                      embedding_size = embedding_size,
                      embedding_weights = embedding_weights,
                      input_length = max_seq_len,
                      hparams = HPARAMS)

trainer(model = model,
        train_dataset = train_dataset, 
        val_dataset = val_dataset,
        hparams = HPARAMS,
        max_epochs = 6,
        verbose = True)

#tf.saved_model.save(model,'data/models/cnn_deep_functional_api.h5',save_format="h5")
tf.keras.models.save_model(model,'lstm_bidirectional.h5',save_format="h5")
new_model = tf.keras.models.load_model('lstm_bidirectional.h5')
print('model saved')

Model: "biLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 50)            46479550  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 20, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 128)           58880     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9    

# Using HParams
Não faz sentido ainda otimizar hiperparâmetros enquanto estivermos com acurácia de quase 100%

In [42]:
# Clear any logs from previous runs
#!rm -rf ./logs/ 

In [43]:
NUM_FILTERS1 = hp.HParam('num_filters1', hp.Discrete([64,128]))#Conforme testes, acima de 128 é melhor
NUM_FILTERS2 = hp.HParam('num_filters2', hp.Discrete([32,64]))
#Conforme testes, acima de 128 é melhor
#Teste com 4 ficou muito abaixo da média

DENSE_UNITS = hp.HParam('dense_units', hp.Discrete([8, 32]))
NUM_WORDS1 = hp.HParam('num_words1', hp.Discrete([3]))  #Conforme testes, acima de 5 é muito ruim
NUM_WORDS2 = hp.HParam('num_words2', hp.Discrete([3]))
HP_DROPOUT = hp.HParam('hp_dropout', hp.RealInterval(0.2,0.3)) 
#0.3 teve resultados piores no passado
#Com poucos parametros, 0.2 estava melhor que 0.3

HP_OPTIMIZER = hp.HParam('hp_optimizer', hp.Discrete(['adam']))# Conforme testes, sgd é ruim,
#'adagrad' e parecido com ADAM
METRIC_ACCURACY = 'accuracy'
USE_SPACY_EMBEDDINGS = False

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[NUM_FILTERS1, NUM_FILTERS2, NUM_WORDS1, NUM_WORDS2, HP_DROPOUT, HP_OPTIMIZER, DENSE_UNITS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        if USE_SPACY_EMBEDDINGS:
            model = selectedModel(embedding_size = embedding_size,
                              input_length = max_seq_len,
                              hparams = {h.name: hparams[h] for h in hparams})
        else:
            model = selectedModel(vocab_size=vocab_size,
                                  embedding_size = embedding_size,
                                  embedding_weights = tokenized_data.embedding_weights,
                                  input_length = max_seq_len,
                                  hparams = {h.name: hparams[h] for h in hparams})
        
        
        hp.hparams(hparams)  # record the values used in this trial
              
        test_accuracy = trainer(model = model,
                                x_train = tokenized_data.x_train,
                                y_train = tokenized_data.y_train,
                                x_test = tokenized_data.x_test,
                                y_test = tokenized_data.y_test,
                                hparams = hparams,
                                max_epochs = 100,
                                verbose = False)
        
        tf.summary.scalar(METRIC_ACCURACY, test_accuracy, step=1)

In [44]:
%%time
print("len(x_train)=",len(tokenized_data.x_train))
print("len(x_test)=",len(tokenized_data.x_test))

session_num = 0
for dense_units in DENSE_UNITS.domain.values:
    for num_filters1 in NUM_FILTERS1.domain.values:
        for num_filters2 in NUM_FILTERS2.domain.values:
            for num_words1 in NUM_WORDS1.domain.values:
                for num_words2 in NUM_WORDS2.domain.values:
                    for hp_dropout in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
                        for hp_optimizer in HP_OPTIMIZER.domain.values:
                            hparams = {
                                    NUM_FILTERS1: num_filters1,
                                    NUM_FILTERS2: num_filters2,
                                    NUM_WORDS1: num_words1,
                                    NUM_WORDS2: num_words2,
                                    HP_DROPOUT: hp_dropout,
                                    HP_OPTIMIZER: hp_optimizer,
                                    DENSE_UNITS: dense_units
                                  }
                            run_name = "run-%d" % session_num
                            print('--- Starting trial: %s' % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            session_num += 1
                            
                            for i in range(5):
                                print(f"i{i}")
                                run('logs/hparam_tuning/' + run_name, hparams)
                            print("---------------------------------------------------------")
                            print("")
print("Training fully completed.")

NameError: ignored

# Funções acessórias

In [ ]:
with open("reservas.txt", "r", encoding="utf8") as f:
  reservas = f.readlines()
reservas = [i.replace('\n','') for i in reservas]
reservas = [i.replace('/',' ') for i in reservas]
reservas = [text_to_word_sequence(frase, filters = '!"“”#$%&()*+-/:=?@[\\]^_`{|}~\t\n', split=' ') for frase in reservas]
new_reservas=[]
for reserva in reservas:
  new_reservas = new_reservas+reserva
new_reservas = list(dict.fromkeys(new_reservas))
with open("palavras_para_acrescentar.txt", "w", encoding="utf8") as f:
  for reserva in new_reservas:
    if not reserva in word_to_index_dict:
      f.write(reserva+'\n')